# Выбор локации для скважины

Для добывающей компании нужно проанализировать данные о скважинах и решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
#pip install pandas-profiling

In [ ]:
import warnings

import pandas as pd
import pandas_profiling
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

from sklearn.metrics import mean_squared_error

In [ ]:
pd.set_option('max_columns', None)

In [ ]:
RANDOM_STATE = 12345

### Загрузим данные

In [ ]:
data = [0, 0, 0]

In [ ]:
data[0] = pd.read_csv('/datasets/geo_data_0.csv')
data[1] = pd.read_csv('/datasets/geo_data_1.csv')
data[2] = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
data[0].shape

(100000, 5)

### Проанализируем данные

In [ ]:
pandas_profiling.ProfileReport(data[0])

In [ ]:
pandas_profiling.ProfileReport(data[1])

In [ ]:
pandas_profiling.ProfileReport(data[2])

In [ ]:
for i in range(3):
    data[i].drop_duplicates(subset=['id'], keep='last', inplace=True)

In [ ]:
for i in range(3):
    data[i] = data[i].drop(columns=['id'])

In [ ]:
data[0].head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


### Вывод

Проанализировав датасеты мы выявили строки с дублирующимися id. Удалили повторяющиеся строки.

0 и 2 регионы имеют слабую корреляцию с параметрами. Аномалии не были обнаружены.

1 регион имеет очень сильную корреляцию параметра f2 и product. Так же 1 регион имеет сильно детерменированное распределение product, что настораживает и требует уточнения у заказчика.

## Обучение и проверка модели

### Разобьем данные на обучающую и валидационную выборки в соотношении 75:25

In [ ]:
def data_split_scale(df):
    target = df['product']
    features = df.drop('product', axis=1)

    #75% - данные для обучения
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=RANDOM_STATE)

    numeric = ['f0', 'f1', 'f2']

    scaler = StandardScaler()
    scaler.fit(features_train[numeric])

    features_train.loc[:, numeric] = scaler.transform(features_train.loc[:, numeric])
    features_valid.loc[:, numeric] = scaler.transform(features_valid.loc[:, numeric])
    return features_train, features_valid, target_train, target_valid

In [ ]:
features_train = [0,0,0]
features_valid = [0,0,0]
target_train = [0,0,0]
target_valid = [0,0,0]

In [ ]:
for i in range(3):
    features_train[i], features_valid[i], target_train[i], target_valid[i] = data_split_scale(data[i])

In [ ]:
features_train[0].count()

f0    74992
f1    74992
f2    74992
dtype: int64

### Обучим модель

In [ ]:
def print_mean_score(scores, i):
    final_score = np.mean(scores)
    print(f'Регион: {i}, Средняя оценка качества модели: {final_score}')

In [ ]:
#подбор параметров
def get_best_estimator(model, params, scoring, features, target, cv):
    GSCV = GridSearchCV(model, params,
                        cv = cv, # количество разбиений на кросс-валидацию
                        scoring = scoring
                       )
    GSCV.fit(features, target)
    print(GSCV.best_estimator_)
    return GSCV.best_estimator_

In [ ]:
warnings.simplefilter(action='ignore')

#### LinearRegression

params = {
    "positive": [True, False],
    "fit_intercept": [True, False]
}

%%time

for i in range(3):    
    model = LinearRegression()
    model = get_best_estimator(model, params, 'accuracy', features_train[i], target_train[i], cv=10)    
    print(f'Регион: {i}, Оценка качества модели: {model.score(features_train[i], target_train[i])}')
    print('')

**Результаты**

LinearRegression(positive=True)
Регион: 0, Оценка качества модели: 0.2534914270257933

LinearRegression(positive=True)
Регион: 1, Оценка качества модели: 0.998792395206123

LinearRegression(positive=True)
Регион: 2, Оценка качества модели: 0.19661169753259355

CPU times: user 4.86 s, sys: 5.07 s, total: 9.92 s
Wall time: 9.91 s

In [ ]:
%%time

for i in range(3):
    model = LinearRegression(positive=True)
    scores = cross_val_score(model, features_train[i], target_train[i], cv=10)
    print_mean_score(scores, i)

Регион: 0, Средняя оценка качества модели: 0.2573906437143596
Регион: 1, Средняя оценка качества модели: 0.9987907508491926
Регион: 2, Средняя оценка качества модели: 0.1994022105120538
CPU times: user 730 ms, sys: 1.15 s, total: 1.88 s
Wall time: 1.83 s


#### Ridge

**Предупреждение**

- Считает долго. Wall time: 5min 26s
- Лучшие параметры: Ridge(alpha=0.1)

params = {
    "alpha": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    "solver": ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
    "random_state": [RANDOM_STATE]
}

%%time

for i in range(3):    
    model = Ridge()
    model = get_best_estimator(model, params, 'accuracy', features_train[i], target_train[i], cv=10)    
    print(f'Регион: {i}, Оценка качества модели: {model.score(features_train[i], target_train[i])}')
    print('')

**Результаты**

Ridge(alpha=0.1, random_state=12345)
Регион: 0, Оценка качества модели: 0.27423906493894545

Ridge(alpha=0.1, random_state=12345)
Регион: 1, Оценка качества модели: 0.9996247760308353

Ridge(alpha=0.1, random_state=12345)
Регион: 2, Оценка качества модели: 0.19661432867295037

CPU times: user 3min 4s, sys: 2min 20s, total: 5min 25s
Wall time: 5min 26s

In [ ]:
%%time

for i in range(3):
    model = Ridge(alpha=0.1, random_state=RANDOM_STATE)
    scores = cross_val_score(model, features_train[i], target_train[i], cv=10)
    print_mean_score(scores, i)

Регион: 0, Средняя оценка качества модели: 0.27788431903763794
Регион: 1, Средняя оценка качества модели: 0.9996227658576522
Регион: 2, Средняя оценка качества модели: 0.19936441960593088
CPU times: user 926 ms, sys: 1.43 s, total: 2.36 s
Wall time: 2.43 s


#### Lasso

**Предупреждение**

- Считает еще дольше, Wall time: 25min 33s
- Лучшие параметры: Lasso(alpha=0.1, max_iter=100, random_state=12345, warm_start=True)


params = {
    "alpha": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    "max_iter": [100, 200, 500, 1000, 1500],
    "selection": ['cyclic', 'random'],
    "warm_start": [True, False],
    "random_state": [RANDOM_STATE]
}

%%time

for i in range(3):    
    model = Lasso()
    model = get_best_estimator(model, params, 'accuracy', features_train[i], target_train[i], cv=10)    
    print(f'Регион: {i}, Оценка качества модели: {model.score(features_train[i], target_train[i])}')
    print('')

**Результаты**

Lasso(alpha=0.1, max_iter=100, random_state=12345, warm_start=True)
Регион: 0, Оценка качества модели: 0.27422686101361304

Lasso(alpha=0.1, max_iter=100, random_state=12345, warm_start=True)
Регион: 1, Оценка качества модели: 0.9996120674602049

Lasso(alpha=0.1, max_iter=100, random_state=12345, warm_start=True)
Регион: 2, Оценка качества модели: 0.19660525632896486

CPU times: user 10min 10s, sys: 15min 16s, total: 25min 27s
Wall time: 25min 33s

In [ ]:
%%time

for i in range(3):
    model = Lasso(alpha=0.1, max_iter=100, random_state=RANDOM_STATE, warm_start=True)
    scores = cross_val_score(model, features_train[i], target_train[i], cv=10)
    print_mean_score(scores, i)

Регион: 0, Средняя оценка качества модели: 0.2778724600519326
Регион: 1, Средняя оценка качества модели: 0.9996099958979604
Регион: 2, Средняя оценка качества модели: 0.1993975005442472
CPU times: user 2.74 s, sys: 6.7 s, total: 9.44 s
Wall time: 9.44 s


#### ElasticNet

**Предупреждение**

- Считает долго. Wall time: 9min 39s
- Лучшие параметры: ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=100, random_state=12345, warm_start=True)

params = {
    "alpha": [0.1, 0.5, 0.9, 1],
    "l1_ratio": [0.1, 0.5, 0.9],
    "max_iter": [100, 500, 1000],
    "warm_start": [True, False],
    "random_state": [RANDOM_STATE]
}

%%time

for i in range(3):    
    model = ElasticNet()
    model = get_best_estimator(model, params, 'accuracy', features_train[i], target_train[i], cv=10)    
    print(f'Регион: {i}, Оценка качества модели: {model.score(features_train[i], target_train[i])}')
    print('')

**Результаты**

ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=100, random_state=12345,
           warm_start=True)
Регион: 0, Оценка качества модели: 0.27244243635748333

ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=100, random_state=12345,
           warm_start=True)
Регион: 1, Оценка качества модели: 0.9927783983567565

ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=100, random_state=12345,
           warm_start=True)
Регион: 2, Оценка качества модели: 0.19525868132892976

CPU times: user 3min 50s, sys: 5min 49s, total: 9min 39s
Wall time: 9min 41s

In [ ]:
%%time

for i in range(3):
    model = ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=100, random_state=RANDOM_STATE, warm_start=True)
    scores = cross_val_score(model, features_train[i], target_train[i], cv=10)
    print_mean_score(scores, i)

Регион: 0, Средняя оценка качества модели: 0.2760768503434289
Регион: 1, Средняя оценка качества модели: 0.9927726309331388
Регион: 2, Средняя оценка качества модели: 0.19800110626006265
CPU times: user 2.4 s, sys: 5.67 s, total: 8.07 s
Wall time: 8.09 s


### Предскажем результаты на валидационной выборке

In [ ]:
predictions_valid = [0, 0, 0]

In [ ]:
for i in range(3):
    features_valid[i] = features_valid[i].reset_index(drop=True)
    target_valid[i] = target_valid[i].reset_index(drop=True)

In [ ]:
for i in range(3):
    model = Ridge(alpha=0.1, random_state=RANDOM_STATE)
    model.fit(features_train[i], target_train[i])
    predictions_valid[i] = pd.DataFrame(model.predict(features_valid[i]))
    rmse = mean_squared_error(target_valid[i], predictions_valid[i], squared=False)
    print(f'Регион: {i}, RMSE модели линейной регрессии на валидационной выборке: {rmse}')

Регион: 0, RMSE модели линейной регрессии на валидационной выборке: 37.926582941523634
Регион: 1, RMSE модели линейной регрессии на валидационной выборке: 0.8877441601060561
Регион: 2, RMSE модели линейной регрессии на валидационной выборке: 40.18554492054703


In [ ]:
for i in range(3):
    print(f'Регион {i}, Средний запас сырья фактически: {target_valid[i].mean()}')
    print(f'Регион {i}, Средний запас предсказанного сырья: {predictions_valid[i][0].mean()}')
    print('')

Регион 0, Средний запас сырья фактически: 92.34784935243168
Регион 0, Средний запас предсказанного сырья: 92.59581526640576

Регион 1, Средний запас сырья фактически: 68.53697799242255
Регион 1, Средний запас предсказанного сырья: 68.52866757783293

Регион 2, Средний запас сырья фактически: 94.56578475991805
Регион 2, Средний запас предсказанного сырья: 94.93593240129654



### Вывод

При обучении модели мы:
- разбили данные 75% train и 25% valid
- стандартизировали параметры f0, f1, f2
- попробовали разные модели с подбором гиперпараметров:
    - LinearRegression
    - Ridge
    - Lasso
    - ElasticNet
- лучше всего отработала модель Ridge с параметрами Ridge(alpha=0.1, random_state=12345)
- сделали кросс-валидацию модели и получили результаты:
    - Регион: 0, Оценка качества модели: 0.27423906493894545
    - Регион: 1, Оценка качества модели: 0.9996247760308353
    - Регион: 2, Оценка качества модели: 0.19661432867295037
- для регионов 0 и 2 все вышеуказанные модели работают плохо и врядли вообще можно корректно их использовать
- для региона 1 почти идеальное предсказание, что тоже странно и надо уточнить данные у заказчика
- предсказали значения для валидационной выборки и посчитали RMSE:
    - Регион: 0, RMSE модели линейной регрессии на валидационной выборке: 37.926582941523634
    - Регион: 1, RMSE модели линейной регрессии на валидационной выборке: 0.8877441601060561
    - Регион: 2, RMSE модели линейной регрессии на валидационной выборке: 40.18554492054703
- для регионов 0 и 2 RMSE большие, для региона 1 почти идеальное
- посчитали среднее значение запаса сырья в регионах
    - Регион 0, Средний запас сырья фактически: 92.34784935243168
    - Регион 0, Средний запас предсказанного сырья: 92.59581526640576

    - Регион 1, Средний запас сырья фактически: 68.53697799242255
    - Регион 1, Средний запас предсказанного сырья: 68.52866757783293

    - Регион 2, Средний запас сырья фактически: 94.56578475991805
    - Регион 2, Средний запас предсказанного сырья: 94.93593240129654
    


## Подготовка к расчёту прибыли

### Создадим константы для значений

In [ ]:
BUDGET = 10 * 10**9 # млрд. р.
PRODUCT_PRICE = 450 * 10**3 # тыс. р.
RISK_PRC = 0.025 # доля, (2.5%)
DRILLHOLE_DEVELOP_COUNT = 200 # Кол-во скважин для рассчета
DRILLHOLE_RESEARCH_COUNT = 500 # Кол-во скважин для исследования

### Посчитаем минимальное среднее кол-во продукта в месторождениях регоина, достаточное для разработки

In [ ]:
product_min = BUDGET / DRILLHOLE_DEVELOP_COUNT / PRODUCT_PRICE
product_min

111.11111111111111

Минимальное для окупаемости кол-во запаса сырья в скважине = 111.11 тыс. баррелей ненфти.

Минимальный запас выше средних значений по региону. Это значит, что есть риск получить убыток.

### Напишем функцию расчета прибыли

In [ ]:
def profit(target, predicted, count):
    predicted_sorted = predicted[0].sort_values(ascending=False)
    selected = target.loc[predicted_sorted.index][:count]
    return PRODUCT_PRICE * selected.sum() - BUDGET

### Вывод

- Создали функцию для расчета выручки.
- Нашли минимальное значение запасов сырья в скважине для окупаемости. Минимальный запас >= 111.(1) тыс тонн нефти.

## Расчёт прибыли и рисков

In [ ]:
state = np.random.RandomState(RANDOM_STATE)

In [ ]:
for i in range(3):
    values = []
    loss_count = 0
    bootstrap_count = 1000

    for _ in range(bootstrap_count):
        target_subsample = target_valid[i].sample(n=DRILLHOLE_RESEARCH_COUNT, replace=True, random_state=state)
        predicted_subsample = predictions_valid[i].loc[target_subsample.index]
        value = profit(target_subsample, predicted_subsample, DRILLHOLE_DEVELOP_COUNT)
        values.append(value)

        if value < 0:
            loss_count += 1

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    mean = values.mean()
    print("Регион:", i)
    print(f"Средняя прибыль: {mean / (10**9)} млрд. р.")
    print(f"Риск убытков: {loss_count / bootstrap_count * 100 } %")
    print(f"2.5%-квантиль: {lower / (10**9)} млрд. р.")
    print(f"97.5%-квантиль: {upper / (10**9)} млрд. р.")
    print('')

Регион: 0
Средняя прибыль: 0.4274421123721394 млрд. р.
Риск убытков: 6.0 %
2.5%-квантиль: -0.12716788234295168 млрд. р.
97.5%-квантиль: 0.9659019834715195 млрд. р.

Регион: 1
Средняя прибыль: 0.5052879672932861 млрд. р.
Риск убытков: 1.0999999999999999 %
2.5%-квантиль: 0.09454465150721354 млрд. р.
97.5%-квантиль: 0.9835746416951212 млрд. р.

Регион: 2
Средняя прибыль: 0.34456337219396027 млрд. р.
Риск убытков: 11.0 %
2.5%-квантиль: -0.23000207476368692 млрд. р.
97.5%-квантиль: 0.9196016737777459 млрд. р.



### Вывод

Изучили финансовые результаты в регионах с использованием обученной модели:

    - с помощью техники bootstrap провели для каждого региона 1000 рассчетов
    - получили следующие результаты:
        - Регион 0:
            - Средняя выручка: 10.42 млрд. р.
            - Риск убытков: 6.0 %
            - 2.5%-квантиль: 9.87 млрд. р.
            - 97.5%-квантиль: 10.96 млрд. р.
        - Регион 1:
            - Средняя выручка: 10.50 млрд. р.
            - Риск убытков: 1.1 %
            - 2.5%-квантиль: 10.09 млрд. р.
            - 97.5%-квантиль: 10.98 млрд. р.
        - Регион 2:
            - Средняя выручка: 10.34 млрд. р.
            - Риск убытков: 11.0 %
            - 2.5%-квантиль: 9.76 млрд. р.
            - 97.5%-квантиль: 10.91 млрд. р.
            
            
    - для разработки следует выбрать регион 1. Он удовлетворяет по окупаемости и имеет значение рисков 1.1% меньше, чем требуется заказчику 2.5%.


## Вывод

В работе нам необходимо было выбрать регион для разработки и добычи нефти.

- Мы изучили и обработали полученные от заказчика данные.
- На основе этих данных обучили модель для предсказания целевого значения - запаса нефти в скважине.
- Наилучшие результаты показала модель с алгоритмом Ridge.
    - Но для регионов 0 и 2 работает плохо.
    - Регион 1 отрабатывает идеально.
- Для каждого региона рассчитали целевые значения используя модель.
- На основе полученных данных рассчитали выручку регионов и риски убытков.
- Лучший результаты были получены в Регионе 1
    - риск убытков = 1.1%, что удовлетворяет требованиям заказчика 2.5%
- Регионы 0 и 1 имеют слишком большие риски убытков

Таким образом для разработки рекумендуется
    